In [261]:
import pyspark.sql.functions
from pyspark.sql import SparkSession
from pyspark.sql.types import *
from Packages import sparkBuilder as sB
import pandas as pd

if __name__ == "__main__":
    spark = sB.build()  #Xây SparkSession thì mới chạy được trên PyCharm

In [262]:
df = spark.read.csv(path="E:\Worksapce\PyCharm\DongPhanTich\data\Data1.csv",header=True,inferSchema=True)
df = df.fillna("Blank")
df.show()

+---------+---------+----------+--------------+-----------+------------+-------------------+-----------------+--------------------+---------------+------+-------------+------+-------------+
|OrderName|CreatedAt|SourceName|TotalDiscounts|OrderStatus|TotalProduct|              Store|CarrierStatusCode|        CancelReason|   SalesProgram|   Sex|        Title| Price|SubtotalPrice|
+---------+---------+----------+--------------+-----------+------------+-------------------+-----------------+--------------------+---------------+------+-------------+------+-------------+
|   100643| 7/1/2023|      Zalo|             0|   prepared|           1|37 Nguyen Van Huyen|         prepared|               Blank|      Dat truoc|Female|       Ao day|180000|       180000|
|   100656| 7/1/2023|  Facebook|             0|     cancel|           1|         Kho Online|           cancel| Khach suy nghi them|          Blank|Female|       Blazer|450000|       450000|
|   100660| 7/1/2023|  Facebook|             0|   

In [263]:
#1. Đếm số lượng mỗi loại sản phẩm bán được
df1 = df.groupBy(df.Title).sum()
df1 = df1.select(df1['Title'],df1['sum(TotalProduct)'].alias('SL'))
df1.show()

+-----------------+----+
|            Title|  SL|
+-----------------+----+
|         Vay body|  95|
|               Vi| 153|
|         Chan vay| 344|
|          Tui deo| 189|
|       Quan jeans| 444|
|           Blazer| 704|
|         Ao phong| 279|
|            Blank|   2|
|               Mu| 168|
|         Quan tay| 732|
|         Vay lien|1135|
|           Hoodie| 234|
|    Quan ong rong| 153|
|        So mi dai| 783|
| Ao croptop so mi| 334|
|That lung dinh da|  96|
|           Ao day| 107|
|           Ao len| 182|
+-----------------+----+



In [264]:
df1.orderBy(df1.SL.desc()).show()
df1.agg({'SL': 'max'}).show()
#1. Đếm số lượng mỗi loại sản phẩm bán được _ END

+-----------------+----+
|            Title|  SL|
+-----------------+----+
|         Vay lien|1135|
|        So mi dai| 783|
|         Quan tay| 732|
|           Blazer| 704|
|       Quan jeans| 444|
|         Chan vay| 344|
| Ao croptop so mi| 334|
|         Ao phong| 279|
|           Hoodie| 234|
|          Tui deo| 189|
|           Ao len| 182|
|               Mu| 168|
|               Vi| 153|
|    Quan ong rong| 153|
|           Ao day| 107|
|That lung dinh da|  96|
|         Vay body|  95|
|            Blank|   2|
+-----------------+----+

+-------+
|max(SL)|
+-------+
|   1135|
+-------+



In [265]:
#2. Đếm số lượng đơn Cancel, Complete
df1 = df.groupBy(df.CarrierStatusCode).count()
df1 = df1.select(df1['CarrierStatusCode'], df1['count'].alias('SL'))

In [266]:
df1.createOrReplaceTempView('sales')
spark.sql('select * from sales where CarrierStatusCode = "cancel" or CarrierStatusCode = "complete"').show()
#2. Đếm số lượng đơn Cancel, Complete _ END

+-----------------+----+
|CarrierStatusCode|  SL|
+-----------------+----+
|           cancel|1891|
|         complete|2602|
+-----------------+----+



In [267]:
#2. Lý giải vì sao Cancel
df.createOrReplaceTempView('sales1')
spark.sql('select OrderName, Title, CarrierStatusCode, CancelReason from sales1 where CarrierStatusCode = "cancel"').show(n= 20,truncate=False)
#2. Lý giải vì sao Cancel _ END

+---------+----------+-----------------+------------------------+
|OrderName|Title     |CarrierStatusCode|CancelReason            |
+---------+----------+-----------------+------------------------+
|100656   |Blazer    |cancel           |Khach suy nghi them     |
|100660   |Blazer    |cancel           |Tra gop khong thanh cong|
|100676   |Chan vay  |cancel           |Don hang spam           |
|103756   |Quan tay  |cancel           |Don trung               |
|103758   |Quan jeans|cancel           |Don trung               |
|103904   |Quan jeans|cancel           |Khach suy nghi them     |
|103937   |Tui deo   |cancel           |Don hang spam           |
|103977   |Tui deo   |cancel           |Het hang                |
|104013   |Ao phong  |cancel           |Don trung               |
|104031   |Quan tay  |cancel           |Don hang spam           |
|104099   |Quan tay  |cancel           |Khach suy nghi them     |
|104151   |Tui deo   |cancel           |Khach suy nghi them     |
|104155   

In [268]:
#3. Đếm số lượng đơn mua tại quầy, đặt trước
df1 = df.groupBy(df.SalesProgram).count()
df1 = df1.select(df1['SalesProgram'], df1['count'].alias('SL'))
df1.createOrReplaceTempView('sales2')
spark.sql('select * from sales2 where SalesProgram = "Mua tai quay" or SalesProgram= "Dat truoc"').show()
#3. Đếm số lượng đơn mua tại quầy, đặt trước _ END

+------------+----+
|SalesProgram|  SL|
+------------+----+
|Mua tai quay|1068|
|   Dat truoc|2230|
+------------+----+



In [269]:
#4. Đếm số lượng khách hàng là nam, nữ kèm tỉ lệ
df1 = df.groupBy(df.Sex).count()
df1 = df1.select(df1['Sex'], df1['count'].alias('SL')).toPandas()

In [270]:
df1["Percentage"] = df1["SL"] * 100 / df1["SL"].sum()
df1["Percentage"]= round(df1["Percentage"],2)
df1 = spark.createDataFrame(df1)
df1.show()
#4. Đếm số lượng khách hàng là nam, nữ kèm tỉ lệ _ END

+------+----+----------+
|   Sex|  SL|Percentage|
+------+----+----------+
|Female|5190|     96.92|
|  Male| 165|      3.08|
+------+----+----------+



In [271]:
#5. Số lượng sản phẩm theo khung giá và tỉ lệ phần trăm
df1 = df.toPandas()
df1

,OrderName,CreatedAt,SourceName,TotalDiscounts,OrderStatus,TotalProduct,Store,CarrierStatusCode,CancelReason,SalesProgram,Sex,Title,Price,SubtotalPrice
0,100643.0,7/1/2023,Zalo,0.0,prepared,1,37 Nguyen Van Huyen,prepared,Blank,Dat truoc,Female,Ao day,180000,180000
1,100656.0,7/1/2023,Facebook,0.0,cancel,1,Kho Online,cancel,Khach suy nghi them,Blank,Female,Blazer,450000,450000
2,100660.0,7/1/2023,Facebook,0.0,cancel,1,37 Nguyen Van Huyen,cancel,Tra gop khong thanh cong,Dat truoc,Female,Blazer,450000,450000
3,100676.0,7/1/2023,Facebook,0.0,cancel,1,Kho Online,cancel,Don hang spam,Blank,Female,Chan vay,310000,310000
4,100706.0,7/1/2023,Store,300000.0,complete,1,37 Nguyen Van Huyen,complete,Blank,Mua tai quay,Female,Blazer,450000,450000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5350,105618.0,9/29/2023,Instagram,0.0,complete,1,Kho Online,complete,Blank,Hang van chuyen,Female,Quan tay,420000,420000
5351,105634.0,9/29/2023,Instagram,0.0,cancel,1,Kho Online,cancel,Khong lien he duoc khach hang,Blank,Female,Mu,180000,180000
5352,105639.0,9/29/2023,Instagram,0.0,complete,1,37 Nguyen Van Huyen,complete,Blank,Dat truoc,Female,Vay lien,430000,430000
5353,105650.0,9/29/2023,Shopee,0.0,cancel,1,37 Nguyen Van Huyen,cancel,Khach khong den,Dat truoc,Female,Quan tay,420000,420000


In [272]:
data = {'Gia_tri': ["So_luong"]}
df2 = pd.DataFrame(data)
df2["<200k"]=df1[df1["Price"]<200000]["TotalProduct"].sum()
df2["200k- <300k"] = df1[df1["Price"] < 300000]["TotalProduct"].sum() - df2["<200k"]
df2["300k- <400k"] = df1[df1["Price"] < 400000]["TotalProduct"].sum() - df2["<200k"] - df2["200k- <300k"]
df2[">=400k"] = df1[df1["Price"]>= 400000]["TotalProduct"].sum()

In [273]:
df2

,Gia_tri,<200k,200k- <300k,300k- <400k,>=400k
0,So_luong,428,620,2281,2805


In [274]:
df1.TotalProduct.sum()

6134

In [275]:
df4= df2.set_index('Gia_tri').T

In [276]:
df4["Percentage"] = df4["So_luong"] * 100 / df4["So_luong"].sum()
df4["Percentage"]= round(df4["Percentage"],2)

In [277]:
df4
#5. Số lượng sản phẩm theo khung giá và tỉ lệ phần trăm _ END

Gia_tri,So_luong,Percentage
<200k,428,6.98
200k- <300k,620,10.11
300k- <400k,2281,37.19
>=400k,2805,45.73


In [278]:
#6. Sắp xếp bảng theo ngày của các tháng, xem tháng nào bán được tổng lượng là bao nhiêu, sp nào nhiều nhất

In [279]:
from pyspark.sql.functions import col
df7 = df.orderBy(df.CreatedAt.asc())

In [280]:
df8 = df7.toPandas()

In [281]:
df8['Month'] = df8['CreatedAt'].str[0:1] #Thêm cột month = cách substring cột order date (2 kí tự đầu)
df=df.dropna()  # Xóa dòng NaN

In [282]:
df8['Month']=df8['Month'].astype(int)  #Chuyển data type từ string sang int
print(set(df8['Month']))
df8

{8, 9, 7}


,OrderName,CreatedAt,SourceName,TotalDiscounts,OrderStatus,TotalProduct,Store,CarrierStatusCode,CancelReason,SalesProgram,Sex,Title,Price,SubtotalPrice,Month
0,100643.0,7/1/2023,Zalo,0.0,prepared,1,37 Nguyen Van Huyen,prepared,Blank,Dat truoc,Female,Ao day,180000,180000,7
1,100656.0,7/1/2023,Facebook,0.0,cancel,1,Kho Online,cancel,Khach suy nghi them,Blank,Female,Blazer,450000,450000,7
2,100660.0,7/1/2023,Facebook,0.0,cancel,1,37 Nguyen Van Huyen,cancel,Tra gop khong thanh cong,Dat truoc,Female,Blazer,450000,450000,7
3,100676.0,7/1/2023,Facebook,0.0,cancel,1,Kho Online,cancel,Don hang spam,Blank,Female,Chan vay,310000,310000,7
4,100706.0,7/1/2023,Store,300000.0,complete,1,37 Nguyen Van Huyen,complete,Blank,Mua tai quay,Female,Blazer,450000,450000,7
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5350,100612.0,9/9/2023,Facebook,0.0,complete,1,Kho Online,complete,Blank,Tu van chuyen,Female,Vay lien,430000,430000,9
5351,100632.0,9/9/2023,Facebook,0.0,cancel,1,Kho Online,cancel,Khach suy nghi them,Blank,Female,Blazer,450000,450000,9
5352,104778.0,9/9/2023,Facebook,0.0,cancel,1,Kho Online,cancel,Don hang spam,Blank,Female,Quan tay,420000,420000,9
5353,104780.0,9/9/2023,Instagram,0.0,confirmed,1,37 Nguyen Van Huyen,confirmed,Blank,Dat truoc,Female,Blazer,450000,450000,9


In [283]:
df1 = df8.groupby('Month')['TotalProduct'].sum().reset_index(name='Orders by months')
df1 = spark.createDataFrame(df1)

In [284]:
df1.orderBy("Orders by months").show()
# Sắp xếp các tháng theo tứ tự tăng dần về số lượng sản phẩm bán được

+-----+----------------+
|Month|Orders by months|
+-----+----------------+
|    9|            1641|
|    8|            2090|
|    7|            2403|
+-----+----------------+

